# `Subquery` vs. `prefetch_related`

In [13]:
mm = TranslatedWork.objects.get(abbreviation='MM')
contents = mm.table_of_contents
# Get the chapter before the last one which has many edits
qs = mm.segments.filter(position__gte=contents[-2]['position'], position__lt=contents[-1]['position'])
#from panta.api.views import TranslatedSegmentViewSet
#view = TranslatedSegmentViewSet()
#view.request = MagicMock()
#qs = view.get_queryset()



In [14]:
%timeit bool(qs.all())

4.55 ms ± 73.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# `Subquery`/`Sum` vs. `Count`/`distinct`

### Some thoughts

Votes are changing all the time. Therefore, I'll probably implement my own caching (in- and decrement the accumulated votes) and read only from the cache. The cache gets updated once a day or week from the database (in case something went wrong). -> Practially, there will be only db hits to write (not to read).

#### With history
1. advantages
    1. `Sum` and `Count` queries are faster with less objects in the table
    2. We still have the user/personal votes. Therefore, it's maybe better to keep the table small.
2. disadvantages
    1. We have two db hits to write with a history
    2. Updating existing votes (possible with a history) results in 2 db hits (with `update_or_create`) in case the user didn't vote yet (which is most of the time)
    3. 3 or 5 db hits to create 1 vote (vs. 1 hit)
3. better update (than creating a new vote every time)
    1. Otherwise 1. check for existing vote, 2. save new vote, 3. create historical record, 4. delete old vote, 5. create historical record
    2. smaller table (= faster queries)
    3. easier calculation
    4. I could add a field to the history to record if it is a revoke
    
### Open questions

1. How are the personal votes stats implemented -> I think I'll cache them
2. How is the votes cache implemented -> I think similar to the current implementation

In [10]:
from django.db import connection, reset_queries
reset_queries()
p = Page.objects.update_or_create(slug='test')
len(connection.queries)

5

In [11]:
connection.queries

[{'sql': 'SELECT "misc_page"."id", "misc_page"."created", "misc_page"."last_modified", "misc_page"."slug", "misc_page"."content", "misc_page"."rendered", "misc_page"."protected" FROM "misc_page" WHERE "misc_page"."slug" = \'test2\' FOR UPDATE',
  'time': '0.001'},
 {'sql': 'SAVEPOINT "s140735693792128_x2"', 'time': '0.000'},
 {'sql': 'INSERT INTO "misc_page" ("created", "last_modified", "slug", "content", "rendered", "protected") VALUES (\'2018-11-15T15:32:32.410467+00:00\'::timestamptz, \'2018-11-15T15:32:32.410491+00:00\'::timestamptz, \'test2\', \'\', \'\', \'{}\') RETURNING "misc_page"."id"',
  'time': '0.000'},
 {'sql': 'INSERT INTO "misc_historicalpage" ("id", "created", "last_modified", "slug", "content", "history_change_reason", "history_date", "history_user_id", "history_type") VALUES (6, \'2018-11-15T15:32:32.410467+00:00\'::timestamptz, \'2018-11-15T15:32:32.410491+00:00\'::timestamptz, \'test2\', \'\', NULL, \'2018-11-15T15:32:32.411113+00:00\'::timestamptz, NULL, \'+\') RE

In [65]:
# Create votes
import random
from panta.constants import ROLES

segments = TranslatedSegment.objects.values_list('id', flat=True)
users = User.objects.values_list('id', flat=True)
votes = []
for i in range(1_000_000):
    votes.append(Vote(
        segment_id=random.choice(segments),
        user_id=random.choice(users),
        role=random.choice(ROLES)[0],
        value=random.choice((-2, -1, 1, 2)),
    ))
result = Vote.objects.bulk_create(votes)

In [16]:
# Get segments
segments = TranslatedSegment.objects.all()
segments = qs

## `Count`

In [59]:
qs1 = segments.annotate(
    translator_approvals=Count(
        'votes',
        filter=Q(votes__role='translator', votes__value=1),
        distinct=True,
    ),
    translator_disapprovals=Count(
        'votes',
        filter=Q(votes__role='translator', votes__value=-1),
        distinct=True,
    ),
    # We wanted to use Exists for disapprovals but it turned out
    # that they influnce each other and there is no distinct attribute
    reviewer_approvals=Count(
        'votes',
        filter=Q(votes__role='reviewer', votes__value=1),
        distinct=True,
    ),
    reviewer_disapprovals=Count(
        'votes',
        filter=Q(votes__role='reviewer', votes__value=-1),
        distinct=True,
    ),
    trustee_approvals=Count(
        'votes',
        filter=Q(votes__role='trustee', votes__value=1),
        distinct=True,
    ),
    trustee_disapprovals=Count(
        'votes',
        filter=Q(votes__role='trustee', votes__value=-1),
        distinct=True,
    ),
)

In [66]:
%timeit bool(qs1.all())

75.3 ms ± 1.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## `Subquery`

In [17]:
from django.db import models

qs2 = segments.annotate(
    translators_acc=Subquery(
        Vote.objects
        .filter(segment=OuterRef('pk'))
        .values('segment_id')
        .annotate(sum=Sum('value', filter=Q(role='translator')))
        .values('sum')[:1]
    ),
    reviewers_acc=Subquery(
        Vote.objects
        .filter(segment=OuterRef('pk'))
        .values('segment_id')
        .annotate(sum=Sum('value', filter=Q(role='reviewer')))
        .values('sum')[:1]
    ),
    trustees_acc=Subquery(
        Vote.objects
        .filter(segment=OuterRef('pk'))
        .values('segment_id')
        .annotate(sum=Sum('value', filter=Q(role='trustee')))
        .values('sum')[:1]
    ),
)
str(qs2.query)

'SELECT "panta_translatedsegment"."id", "panta_translatedsegment"."created", "panta_translatedsegment"."last_modified", "panta_translatedsegment"."position", "panta_translatedsegment"."page", "panta_translatedsegment"."tag", "panta_translatedsegment"."classes", "panta_translatedsegment"."content", "panta_translatedsegment"."reference", "panta_translatedsegment"."work_id", "panta_translatedsegment"."original_id", "panta_translatedsegment"."locked_by_id", "panta_translatedsegment"."progress", (SELECT SUM(U0."value") FILTER (WHERE U0."role" = translator) AS "sum" FROM "panta_vote" U0 WHERE U0."segment_id" = ("panta_translatedsegment"."id") GROUP BY U0."segment_id" LIMIT 1) AS "translators_acc", (SELECT SUM(U0."value") FILTER (WHERE U0."role" = reviewer) AS "sum" FROM "panta_vote" U0 WHERE U0."segment_id" = ("panta_translatedsegment"."id") GROUP BY U0."segment_id" LIMIT 1) AS "reviewers_acc", (SELECT SUM(U0."value") FILTER (WHERE U0."role" = trustee) AS "sum" FROM "panta_vote" U0 WHERE U0.

In [67]:
%timeit bool(qs2.all())

41.5 ms ± 523 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [41]:
result = 0
for v in Vote.objects.all():
    result += v.value
    
result

-32